# Comparing datasets

In [ ]:
import os; from os import listdir; from os.path import isfile, join
import re  
#from skimage import io 
from skimage.io import imread
# To manipulate arrays
import numpy as np 
import random
from tqdm.notebook import tqdm
from timeit import default_timer as timer
import scipy
import pandas as pd
import shutil
import pathlib
import sys
# Plotting
import matplotlib as mpl ; import matplotlib.pyplot as plt 
# To create interactive elements
import ipywidgets as widgets ; from ipywidgets import interact, interactive, fixed, interact_manual, Button, HBox, VBox, Layout, GridspecLayout ; from IPython.display import Image

In [ ]:
# Deffining directories
current_dir = pathlib.Path().absolute()
sequences_dir = current_dir.parents[1].joinpath('DataBases','gene_files')
video_dir = current_dir.parents[1].joinpath('DataBases','videos_for_sim_cell')
trajectiories_dir = current_dir.parents[1].joinpath('DataBases','rsnapsim_simulations','bactin_ssa.npy')
rsnaped_dir = current_dir.parents[1].joinpath('rsnaped')

In [ ]:
# Importing rSNAPsim_IP
sys.path.append(str(rsnaped_dir))
import rsnaped as rsp

In [ ]:
# These are the parameters that need to be tested. 
number_of_simulated_cells = 1  # PLEASE TEST MIN 1 MAX 10
number_spots_per_cell = 50     # PLEASE TEST MIN 5 MAX 200
simulation_time_in_sec = 100     # PLEASE TEST MIN 10 MAX 100
diffusion_coefficient = 0.56    # PLEASE TEST MIN 0.1 MAX 2

In [ ]:
intensity_calculation_method = 'disk_donut'  # options are : 'total_intensity' and 'disk_donut' 'gaussian_fit'
mask_selection_method = 'max_area' # options are : 'max_spots' and 'max_area' 
use_optimization_for_tracking = 0 # 0 not using, 1 is using optimization
frame_selection_empty_video = 'shuffle' # Options are: 'constant' , 'shuffle' and 'loop'

In [ ]:
particle_size = 5 # spot size for the simulation and tracking.
elongation_rate = 3
initiation_rate = 0.03

In [ ]:
def fun_simulated_cells(current_dir,trajectiories_dir, video_dir,number_of_simulated_cells=3,number_spots_per_cell=80,simulation_time_in_sec =100,step_size_in_sec=1,particle_size=5, diffusion_coefficient =1, path_to_save_output='temp',intensity_calculation_method='gaussian_fit',frame_selection_empty_video=frame_selection_empty_video):
    spot_size = particle_size
    spot_sigma = 2
    # Code that creates the folder to store results.
    diffusion_coefficient_string = str(diffusion_coefficient).replace('.','_')
    
    directory_name = 'Simulation_V2__'+'ns_'+str(number_spots_per_cell) +'_diff_'+ diffusion_coefficient_string 
    path_to_save_output = 'temp'
    save_to_path =  current_dir.joinpath(path_to_save_output , directory_name )
    
    if not os.path.exists(str(save_to_path)):
        os.makedirs(str(save_to_path))
    else:
        shutil.rmtree(str(save_to_path))
        os.makedirs(str(save_to_path))
        
    # Loading trajectories from file
    ssa_trajectories = np.load(str(trajectiories_dir))
    counter = 0
    ## Main loop that creates each cell and dataframe
    for cell_number in range (0, number_of_simulated_cells):
        ouput_directory_name = str(video_dir)
        list_files_names = sorted([f for f in listdir(ouput_directory_name) if isfile(join(ouput_directory_name, f)) and ('.tif') in f], key=str.lower)  # reading all tif files in the folder
        list_files_names.sort(key=lambda f: int(re.sub('\D', '', f)))  # sorting the index in numerical order
        path_files = [ str(video_dir.joinpath(f).resolve()) for f in list_files_names ] # creating the complete path for each file
        video_path = path_files[counter]        
        video = imread(video_path) 
        counter +=1
        if counter>=len(path_files):
            counter =0
        random_index_ch1 = np.random.randint(low=0, high=ssa_trajectories.shape[0]-1, size=(number_spots_per_cell,))
        random_index_ch2 = np.random.randint(low=0, high=ssa_trajectories.shape[0]-1, size=(number_spots_per_cell,))
        simulated_trajectories_ch1 = ssa_trajectories[random_index_ch1,0:simulation_time_in_sec:step_size_in_sec]
        simulated_trajectories_ch2 =  ssa_trajectories[random_index_ch2,0:simulation_time_in_sec:step_size_in_sec]
        # Running the cell simulation
        saved_file_name = str(save_to_path.joinpath('sim_cell_'+str(cell_number)))
        tensor_video , spot_positions_movement, DataFrame_particles_intensities = rsp.SimulatedCell( base_video=video, number_spots = number_spots_per_cell, number_frames=simulation_time_in_sec, step_size=step_size_in_sec, diffusion_coefficient =diffusion_coefficient, simulated_trajectories_ch0=None, size_spot_ch0=spot_size, spot_sigma_ch0=spot_sigma, simulated_trajectories_ch1=simulated_trajectories_ch1, size_spot_ch1=spot_size, spot_sigma_ch1=spot_sigma, simulated_trajectories_ch2=simulated_trajectories_ch2, size_spot_ch2=spot_size, spot_sigma_ch2=spot_sigma, ignore_ch0=0,ignore_ch1=0, ignore_ch2=1,save_as_tif_uint8=0,save_as_tif =1,save_as_gif=0, save_dataframe=1, saved_file_name=saved_file_name,create_temp_folder = False, intensity_calculation_method=intensity_calculation_method,perform_video_augmentation=1,frame_selection_empty_video=frame_selection_empty_video).make_simulation()      
        #print ('The results are saved in folder: ', saved_file_name)
    return save_to_path

In [ ]:
ouput_directory_name = fun_simulated_cells(current_dir,trajectiories_dir,video_dir,number_of_simulated_cells=number_of_simulated_cells,number_spots_per_cell=number_spots_per_cell,simulation_time_in_sec =simulation_time_in_sec,step_size_in_sec=1,particle_size=particle_size, diffusion_coefficient=diffusion_coefficient,intensity_calculation_method=intensity_calculation_method,frame_selection_empty_video=frame_selection_empty_video)

In [ ]:
# Reads the folder with the results and import the simulations as lists
list_files_names = sorted([f for f in listdir(ouput_directory_name) if isfile(join(ouput_directory_name, f)) and ('.tif') in f], key=str.lower)  # reading all tif files in the folder
list_files_names.sort(key=lambda f: int(re.sub('\D', '', f)))  # sorting the index in numerical order
path_files = [ str(ouput_directory_name.joinpath(f).resolve()) for f in list_files_names ] # creating the complete path for each file

# Reading the microscopy data
list_videos = [imread(f)[:,:,:,:] for f in  path_files] # List with all the videos
nimg = number_of_simulated_cells

In [ ]:
# Reads the dataframes
list_files_dfnames = sorted([f for f in listdir(ouput_directory_name) if isfile(join(ouput_directory_name, f)) and ('.csv') in f], key=str.lower)  # reading all tif files in the folder
list_files_dfnames.sort(key=lambda f: int(re.sub('\D', '', f)))  # sorting the index in numerical order
path_files_df = [ str(ouput_directory_name.joinpath(f).resolve()) for f in list_files_dfnames ] # creating the complete path for each file
list_df_real_positions = [pd.read_csv(f) for f in  path_files_df] # List with all the dataframes

In [ ]:
rsp.VisualizerImage(list_videos,list_files_names=list_files_names,selected_channel =0,selected_timepoint= 0,normalize=0,individual_figure_size=7).plot()

In [ ]:
#intensity_selection_method =intensity_selection_method, mask_selection_method = mask_selection_method
list_DataFrame_particles_intensities= []
list_array_intensities = []
list_time_vector = []
for i in tqdm(range(0,nimg)): 
    DataFrame_particles_intensities, array_intensities, time_vector, mean_intensities,std_intensities, mean_intensities_normalized, std_intensities_normalized = rsp.PipelineTracking(list_videos[i],particle_size=particle_size,file_name=list_files_names[i],selected_channel=0,intensity_calculation_method =intensity_calculation_method, mask_selection_method = mask_selection_method,show_plot=1,use_optimization_for_tracking=use_optimization_for_tracking,real_positions_dataframe = list_df_real_positions[i],average_cell_diameter=200,print_process_times=1).run()    
    list_DataFrame_particles_intensities.append(DataFrame_particles_intensities)
    list_array_intensities.append(array_intensities)
    list_time_vector.append(time_vector)

In [ ]:
# Dataframe from simulated data
num_particles_real = list_df_real_positions[0]['particle'].nunique()
print('number of particles:', num_particles_real , '\n')
list_df_real_positions[0]

In [ ]:
# Dataframe from tracking
num_particles_tracking = list_DataFrame_particles_intensities[0]['particle'].nunique()
print('number of particles:',num_particles_tracking, '\n')
list_DataFrame_particles_intensities[0]

In [ ]:
# https://stackoverflow.com/questions/31667070/max-distance-between-2-points-in-a-data-set-and-identifying-the-points/31667270#31667270
# https://stackoverflow.com/questions/31632190/measuring-geographic-distance-with-scipy
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.pdist.html#scipy.spatial.distance.pdist

## Metric to calculate the distance between trajectories

In [ ]:
# https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.103.2617&rep=rep1&type=pdf

Euclidean distance is used to calculate the distance between each point

$d=(x_2−x_1)^2+(y_2−y_1)^2$

Average

$\mu(di) = \frac{1}{n} \sum\limits_{i=1}^n = d_i$

Standard deviation

$\sigma = \sqrt{\frac{1}{N} \sum_{i=1}^N (x_i - \mu)^2},\text{ where } \mu = \frac{1}{N} \sum_{i=1}^N x_i$

Standard error of the sample mean

$SEM = \frac{\sigma_{x}}{\sqrt{N}}$

In [ ]:
import itertools
from scipy.spatial.distance import pdist
from scipy import stats

df_sim = list_df_real_positions[0]
df_tracking = list_DataFrame_particles_intensities[0]

list_idx_sim = np.arange(0,num_particles_real,step=1)
list_idx_tracking = np.arange(0,num_particles_tracking,step=1)

distance_matrix_mean = np.zeros(( len(list_idx_sim),len(list_idx_tracking)))
distance_matrix_sem = np.zeros(( len(list_idx_sim),len(list_idx_tracking)))

# Loop that iterates for all trajectory combinations
for index_sim, index_tracking in itertools.product(list_idx_sim,list_idx_tracking):
    time_df_tracking = df_tracking.loc[df_tracking['particle']==index_tracking].frame.values[:]
    temp_distance_matrix = []
    # Loop that iterates for the length of the tracking trajectory
    for index_time in time_df_tracking:
        # coordinates simulation
        #print(index_sim,index_time)
        x_val_sim = df_sim.loc[(df_sim['particle']==index_sim) & (df_sim['frame']==index_time)].x.values[0]
        y_val_sim = df_sim.loc[(df_sim['particle']==index_sim) & (df_sim['frame']==index_time)].y.values[0]
        # coordinates tracking
        x_val_tracking = df_tracking.loc[(df_tracking['particle']==index_tracking) & (df_tracking['frame']==index_time)].x.values[0]
        y_val_tracking = df_tracking.loc[(df_tracking['particle']==index_tracking) & (df_tracking['frame']==index_time)].y.values[0]        
        # Combining coordinates between simulation and tracking
        coordinates_sim = [x_val_sim,y_val_sim]
        coordinates_tracking = [x_val_tracking,y_val_tracking]
        matrix_coordinates = [coordinates_sim, coordinates_tracking] 
        # Calculating the Euclidean distance
        temp_distance_matrix.append( np.round(pdist(matrix_coordinates, 'euclidean'),2) )
    # Saving comparison in a numpy array
    distance_matrix_mean[index_sim, index_tracking] = np.mean(temp_distance_matrix)
    distance_matrix_sem[index_sim, index_tracking] = stats.sem(temp_distance_matrix, axis=None)

In [ ]:
distance_matrix_mean.shape

In [ ]:
print(len(list_idx_sim),len(list_idx_tracking))

In [ ]:
# Selecting the closest trajectories
min_distances = np.amin (distance_matrix_mean,axis =0)
print(np.round(min_distances,1))

In [ ]:
# Selecting the closest trajectories
min_distances = np.amin (distance_matrix_sem,axis =0)
print(np.round(min_distances,1))

In [ ]:
# making pairs if the distance is shorter than a treshold
min_distances = np.amin (distance_matrix_mean,axis =0)
min_index_distances = np.argmin (distance_matrix_mean,axis =0)
pairs = []
false_detection = []
treshold_mean = 2
list_selected_trajectories =[]

# loop that detect paired trajectories and false trajectories
for i in list_idx_tracking:
    if min_distances[i] < treshold_mean:
        pairs.append([min_index_distances[i],i])
        list_selected_trajectories.append(i)
    else:
        false_detection.append(i)

In [ ]:
print(pairs)

In [ ]:
print(false_detection)

In [ ]:
print(list_selected_trajectories)
fragmented_trejectories =  [list_selected_trajectories[i] for i in range(len(list_selected_trajectories)) if list_selected_trajectories[i] in list_selected_trajectories[:i]][1:]
fragmented_trejectories

In [ ]:
selected_pair = 10

# coordinates tracking
x_val_sim = df_sim.loc[df_sim['particle']==pairs[selected_pair][0] ].x.values[:]
y_val_sim = df_sim.loc[df_sim['particle']==pairs[selected_pair][0] ].y.values[:]

x_val_tracking = df_tracking.loc[df_tracking['particle']==pairs[selected_pair][1] ].x.values[:]
y_val_tracking = df_tracking.loc[df_tracking['particle']==pairs[selected_pair][1] ].y.values[:]   

plt.figure(figsize=(10,10))
plt.plot(x_val_sim,y_val_sim, '-',linewidth = 3,color='lime');
plt.plot(x_val_tracking,y_val_tracking, '-',linewidth = 3,color='orangered');

plt.title('Comparing trajectories')
plt.show()

In [ ]:
import math
from matplotlib import gridspec

NUM_COLUMNS =3
NUM_PLOTS = len(pairs)
NUM_ROWS = int(math.ceil(NUM_PLOTS/ NUM_COLUMNS))
FIGURE_SIZE = 5

# Loop to plot multiple cells in a grid
gs = gridspec.GridSpec(NUM_ROWS, NUM_COLUMNS)
gs.update(wspace=0.2, hspace=0.2) # set the spacing between axes.
fig = plt.figure(figsize=(FIGURE_SIZE*NUM_COLUMNS, FIGURE_SIZE*NUM_ROWS))
for i in range(0,NUM_PLOTS):    
    
    # coordinates tracking
    x_val_sim = df_sim.loc[df_sim['particle']==pairs[i][0] ].x.values[:]
    y_val_sim = df_sim.loc[df_sim['particle']==pairs[i][0] ].y.values[:]
    x_val_tracking = df_tracking.loc[df_tracking['particle']==pairs[i][1] ].x.values[:]
    y_val_tracking = df_tracking.loc[df_tracking['particle']==pairs[i][1] ].y.values[:]   

    ax = fig.add_subplot(gs[i])
    ax.plot(x_val_sim,y_val_sim, '-',linewidth = 3,color='lime');
    ax.plot(x_val_tracking,y_val_tracking, '-',linewidth = 3,color='orangered');
    
    ax.set(title='sim:'+ str(pairs[i][0])+', tracking: ' + str(pairs[i][1]) )
    